In [6]:
!pip install langchain_gigachat pandas numpy scikit-learn python-dotenv

In [7]:
from dotenv import load_dotenv

load_dotenv()


True

In [8]:
from giga_agent.utils.llm import load_embeddings

emb = load_embeddings()

In [9]:
import pandas as pd

df = pd.read_csv('rusentiment_random_posts.csv')

ModuleNotFoundError: No module named 'pandas'

In [6]:
filtered_df = pd.concat([df[df['label'] == 'neutral'][:1500], df[df['label'] == 'positive'][:1200], df[df['label'] == 'negative'][:1000]])

In [7]:
embs = emb.embed_documents(list(filtered_df['text']))

In [8]:
len(embs)

3700

In [9]:
filtered_df['emb'] = embs

In [10]:
filtered_df

,label,text,emb
5,neutral,Решите уравнение мужских дел для жизни :\nДомо...,"[-1.837890625, 0.226318359375, 1.5, 1.11816406..."
7,neutral,Просто пост :),"[-5.0859375, -0.6455078125, 3.802734375, 1.769..."
14,neutral,познакомил свою девушку с Ксюшей,"[-3.009765625, 1.0556640625, 2.669921875, 1.41..."
22,neutral,"\nРекомендую смотреть это видео, раскрыв на ве...","[-2.45703125, -1.396484375, 4.67578125, 1.7392..."
27,neutral,Не позволяйте женщине быть сильной… Ведь силой...,"[-4.68359375, -0.9248046875, 6.07421875, 0.038..."
...,...,...,...
8497,negative,Сегодня в с.Петровское Беляевского района прои...,"[1.1845703125, 0.2646484375, 2.615234375, 0.14..."
8520,negative,"Пидор,гамосек,параститутка","[-2.03125, 0.1951904296875, 3.71875, 1.640625,..."
8553,negative,"Обманул раз, обманет и другой! Ударил раз, уда...","[-3.630859375, 1.3017578125, 3.48046875, -0.10..."
8571,negative,не ты охренела или что?),"[-3.013671875, -0.0207061767578125, 3.31640625..."


In [11]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = np.vstack(filtered_df["emb"].values)
y = filtered_df["label"].values

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

clf = LogisticRegression(max_iter=1000, n_jobs=-1, multi_class="ovr", class_weight={'neutral':1.2, 'positive':1.0, 'negative':1})
clf.fit(X_train, y_train)

print(classification_report(y_val, clf.predict(X_val)))

/Users/mikelarg/.pyenv/versions/3.12.9/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


              precision    recall  f1-score   support

    negative       0.76      0.77      0.76       200
     neutral       0.76      0.76      0.76       300
    positive       0.77      0.77      0.77       240

    accuracy                           0.77       740
   macro avg       0.77      0.77      0.77       740
weighted avg       0.77      0.77      0.77       740



In [12]:
def predict_sentiment(text: str) -> str:
    emb_ = np.array(emb.embed_query(text)).reshape(1, -1)
    proba = clf.predict_proba(emb_)[0]      # или model.predict(emb)
    return clf.classes_[proba.argmax()] 

In [13]:
!pip install joblib

In [14]:
import joblib

# clf — ваш обученный LogisticRegression
joblib.dump(clf, "sentiment_logreg.joblib", compress=3)

['sentiment_logreg.joblib']

In [16]:
predict_sentiment("очень классный товар")

'positive'